In [ ]:
%reset -f

In [ ]:
%pwd

In [5]:
%%javascript
document.addEventListener('keydown', function(e) {
  if (e.key === 'F12') {
    e.preventDefault()
    if (document.fullscreenElement) {
      document.exitFullscreen();
    } else {
      document.documentElement.requestFullscreen();
    }
  }
});


<IPython.core.display.Javascript object>

In [1]:
import pandas as pd
import arcpy
import os

myProject = 'MyProject1'
projDIR = r'C:\Users\fullh\Documents\ArcGIS\Projects\{}'.format(myProject)
dataDIR = r'C:\Users\fullh\Documents\ArcGIS\Projects\{}\Data'.format(myProject)
gdbDIR = r'C:\Users\fullh\Documents\ArcGIS\Projects\{}\{}.gdb'.format(myProject, myProject)
arcpy.env.overwriteOutput = True

data_files = os.listdir(dataDIR)

input_string = ';'.join(data_files)
input_list = input_string.split(';')
band_n = len(input_list)
print(band_n)

32


In [10]:

arcpy.env.workspace = dataDIR

with arcpy.EnvManager(scratchWorkspace=gdbDIR):
  output_raster = arcpy.ia.RasterCalculator(
    rasters = ['CCSM4_1974_2013_ero.tif'],
    input_names = ['raster'],
    expression = 'raster >= 0'
  )
output_raster.save(os.path.join(gdbDIR, 'clip_raster'))

with arcpy.EnvManager(extent='MAXOF'):
  arcpy.management.CompositeBands(
    in_rasters = input_string,
    out_raster = os.path.join(gdbDIR, 'CompositeBands')
  )

arcpy.env.workspace = gdbDIR

with arcpy.EnvManager(outputZFlag='Disabled', outputMFlag='Disabled'):
  arcpy.conversion.RasterToPolygon(
    in_raster = 'clip_raster',
    out_polygon_features = os.path.join(gdbDIR, 'clip_shape'),
    simplify = 'SIMPLIFY',
    raster_field = 'Value',
    create_multipart_features = 'SINGLE_OUTER_PART',
    max_vertices_per_feature = None
  )

with arcpy.EnvManager(scratchWorkspace=gdbDIR):
  output_raster = arcpy.ia.RasterCalculator(
    rasters = ['CompositeBands'],
    input_names = ['raster'],
    expression = 'raster * 1'
  )
output_raster.save(os.path.join(gdbDIR, 'NoData_Composite'))

with arcpy.EnvManager(scratchWorkspace=gdbDIR):
  output_raster = arcpy.ia.RasterCalculator(
    rasters = ['NoData_Composite'],
    input_names = ['raster'],
    expression = 'Con(IsNull(raster),1,raster)'
  )
output_raster.save(os.path.join(gdbDIR, 'Filled_Composite'))

with arcpy.EnvManager(scratchWorkspace=gdbDIR):
  output_raster = arcpy.sa.ExtractByMask(
    in_raster = 'Filled_Composite',
    in_mask_data = 'clip_shape',
    extraction_area = 'INSIDE',
    analysis_extent = '-120.004083333317 31.3333333333001 -102.995916734683 42.004083290617 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'
  )
output_raster.save(os.path.join(gdbDIR, 'Extract_Composite'))

with arcpy.EnvManager(scratchWorkspace=projDIR):
  out_multiband_raster = arcpy.sa.PrincipalComponents(
    in_raster_bands = 'Extract_Composite',
    number_components = band_n,
    out_data_file = 'pca_info.txt'
  )
out_multiband_raster.save(os.path.join(gdbDIR, 'Princip_Composite'))

print('DONE')


DONE


In [ ]:

with open(os.path.join(projDIR, 'pca_info.txt')) as f:
  lines = f.readlines()

for i, line in enumerate(lines):
  if 'CORRELATION MATRIX' in line:
    hdr_line_idx = i
    for j, l in enumerate(lines[hdr_line_idx:]):
      if '#  ========' in l:
        ftr_line_idx = hdr_line_idx + j
        break
    break

start_idx = hdr_line_idx + 4
end_idx = ftr_line_idx - 1

corr_m_lines = lines[start_idx:end_idx + 1]

corr_data = []
for l in corr_m_lines:
  row = l.split()
  corr_data.append([float(x) for x in row[1:]])

corr_df = pd.DataFrame(corr_data)

def vif_fn(x):
  if x != 1:
    vif = (1)/(1 - x**2)
  else:
    vif = 0
  return vif

vif_df = corr_df.map(vif_fn)

high_value_mask = vif_df >= 5
stacked_mask = high_value_mask.stack()
result = stacked_mask[stacked_mask]

matrix_index_col_list = []
for (index, col), _ in result.items():
  matrix_index_col_list.append([index, col])

index_col_list = []
for l in matrix_index_col_list:
  if l not in index_col_list and l.reverse() not in index_col_list:
    index_col_list.append(l)

print('VIF FACTORS >=5:')  
for index, col in index_col_list:
  print(f"Index: {index}, Column: {col}, Value: {vif_df.loc[index, col]}")
  print(input_list[index], input_list[col])

print(len(index_col_list))

In [3]:
removal_choices = ['pmomx_1974_2013.tif', 'CCSM4_1974_2013_pet.tif', 'DEM_Elev.tif', 'POL_theta_s.tif', 'POL_theta_r.tif', 'RAP_afg.tif', 'RAP_pfg.tif', 'RAP_tre.tif', 'tmean_1974_2013.tif']
print(removal_choices)
print(len(removal_choices))

uncorr_input_list = []
for l in input_list:
  if l not in removal_choices:
    uncorr_input_list.append(l)

uncorr_input_string = ';'.join(uncorr_input_list)
band_n = len(uncorr_input_list)
print(uncorr_input_list)
print(band_n)


['pmomx_1974_2013.tif', 'CCSM4_1974_2013_pet.tif', 'DEM_Elev.tif', 'POL_theta_s.tif', 'POL_theta_r.tif', 'RAP_afg.tif', 'RAP_pfg.tif', 'RAP_tre.tif', 'tmean_1974_2013.tif']
9
['accm_1974_2013.tif', 'CCSM4_1974_2013_consec.tif', 'CCSM4_1974_2013_ero.tif', 'CCSM4_1974_2013_ndays.tif', 'CCSM4_1974_2013_swe.tif', 'DEM_Slope.tif', 'pmomn_1974_2013.tif', 'POL_bd.tif', 'POL_clay.tif', 'POL_ksat.tif', 'POL_om.tif', 'POL_ph.tif', 'POL_sand.tif', 'POL_silt.tif', 'RAP_afgNPP.tif', 'RAP_bgr.tif', 'RAP_ltr.tif', 'RAP_pfgNPP.tif', 'RAP_shr.tif', 'RAP_shrNPP.tif', 'RAP_treNPP.tif', 'tmomn_1974_2013.tif', 'tmomx_1974_2013.tif']
23


In [2]:

#Iso Cluster using PCA as input and multidimensional raster.


#input_string = uncorr_input_string

arcpy.env.workspace = dataDIR

with arcpy.EnvManager(extent='MAXOF'):
  arcpy.management.CompositeBands(
    in_rasters = input_string,
    out_raster = os.path.join(gdbDIR, 'CompositeBands')
  )

arcpy.env.workspace = gdbDIR

with arcpy.EnvManager(scratchWorkspace=gdbDIR):
  output_raster = arcpy.ia.RasterCalculator(
    rasters = ['CompositeBands'],
    input_names = ['raster'],
    expression = 'raster * 1'
  )
output_raster.save(os.path.join(gdbDIR, 'NoData_Composite'))

with arcpy.EnvManager(scratchWorkspace=gdbDIR):
  output_raster = arcpy.ia.RasterCalculator(
    rasters = ['NoData_Composite'],
    input_names = ['raster'],
    expression = 'Con(IsNull(raster),1,raster)'
  )
output_raster.save(os.path.join(gdbDIR, 'Filled_Composite'))

with arcpy.EnvManager(scratchWorkspace=gdbDIR):
  output_raster = arcpy.sa.ExtractByMask(
    in_raster = 'Filled_Composite',
    in_mask_data = 'clip_shape',
    extraction_area = 'INSIDE',
    analysis_extent = '-120.004083333317 31.3333333333001 -102.995916734683 42.004083290617 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'
  )
output_raster.save(os.path.join(gdbDIR, 'Extract_Composite'))

arcpy.md.RasterToNetCDF(
  in_raster = 'Extract_Composite',
  out_netCDF_file = os.path.join(projDIR, 'RasterToNetCD.nc'),
  variable = 'Extract_Composite',
  variable_units = '',
  x_dimension = 'lon',
  y_dimension = 'lat',
  band_dimension = 'band',
  fields_to_dimensions = None,
  compression_level = 0
)

arcpy.md.MakeMultidimensionalRasterLayer(
  in_multidimensional_raster=os.path.join(projDIR, 'RasterToNetCD.nc'),
  out_multidimensional_raster_layer='MultidimRasterLyr',
  variables='Extract_Composite',
  dimension_def='ALL',
  dimension_ranges=None,
  dimension_values=None,
  dimension='',
  start_of_first_iteration='',
  end_of_first_iteration='',
  iteration_step=None,
  iteration_unit='',
  template='-120.00833328533 31.3333333207995 -102.991666625469 42.00833331653 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
  dimensionless='DIMENSIONS',
  spatial_reference=None
)

arcpy.ia.MultidimensionalPrincipalComponents(
  in_multidimensional_raster='MultidimRasterLyr',
  mode='DIMENSION_REDUCTION',
  dimension='band',
  out_pc=os.path.join(projDIR, 'MultidimRasterLyr_d_pc.crf'),
  out_loadings=os.path.join(projDIR, 'MultidimRasterLyr_d_loading.csv'),
  out_eigenvalues=os.path.join(gdbDIR, 'eigenvalues'),
  variable='Extract_Composite',
  number_of_pc='99%'
)

with arcpy.EnvManager(scratchWorkspace=os.path.join(gdbDIR)):
  Output_classified_raster = arcpy.sa.IsoClusterUnsupervisedClassification(
    in_raster_bands=os.path.join(projDIR, 'MultidimRasterLyr_d_pc.crf'),
    Number_of_classes=20,
    Minimum_class_size=20,
    Sample_interval=10,
    out_signature_file=os.path.join(projDIR, 'iso_signature.gsg')
  )
  Output_classified_raster.save(os.path.join(gdbDIR, 'IsoCluster'))

print('DONE')

# arcpy.md.MakeNetCDFRasterLayer(
#   in_netCDF_file = os.path.join(projDIR, 'RasterToNetCD.nc'),
#   variable = 'Extract_Composite',
#   x_dimension = 'lon',
#   y_dimension = 'lat',
#   out_raster_layer = 'netCDFlyr',
#   band_dimension = 'band',
#   dimension_values = None,
#   value_selection_method = 'BY_VALUE',
#   cell_registration = 'CENTER'
# )




DONE


In [ ]:
Extract_Composite -> netCDF (via Raster to NetCDF) -> netCDF lyr (via Make NetCDF Raster Layer) -> multidim raster lyr (via Make Multidimensional Raster Layer) -> pca output (via Multidimensional Principle Components)

In [8]:

#Iso Cluster using composite as input.


#input_string = uncorr_input_string

arcpy.env.workspace = dataDIR

with arcpy.EnvManager(extent='MAXOF'):
  arcpy.management.CompositeBands(
    in_rasters = input_string,
    out_raster = os.path.join(gdbDIR, 'CompositeBands')
  )

arcpy.env.workspace = gdbDIR

with arcpy.EnvManager(scratchWorkspace=gdbDIR):
  output_raster = arcpy.ia.RasterCalculator(
    rasters = ['CompositeBands'],
    input_names = ['raster'],
    expression = 'raster * 1'
  )
output_raster.save(os.path.join(gdbDIR, 'NoData_Composite'))

with arcpy.EnvManager(scratchWorkspace=gdbDIR):
  output_raster = arcpy.ia.RasterCalculator(
    rasters = ['NoData_Composite'],
    input_names = ['raster'],
    expression = 'Con(IsNull(raster),1,raster)'
  )
output_raster.save(os.path.join(gdbDIR, 'Filled_Composite'))

with arcpy.EnvManager(scratchWorkspace=gdbDIR):
  output_raster = arcpy.sa.ExtractByMask(
    in_raster = 'Filled_Composite',
    in_mask_data = 'clip_shape',
    extraction_area = 'INSIDE',
    analysis_extent = '-120.004083333317 31.3333333333001 -102.995916734683 42.004083290617 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'
  )
output_raster.save(os.path.join(gdbDIR, 'Extract_Composite'))

with arcpy.EnvManager(scratchWorkspace=os.path.join(gdbDIR)):
  Output_classified_raster = arcpy.sa.IsoClusterUnsupervisedClassification(
    in_raster_bands='Extract_Composite',
    Number_of_classes=20,
    Minimum_class_size=20,
    Sample_interval=10,
    out_signature_file=os.path.join(projDIR, 'iso_signature.gsg')
  )
  Output_classified_raster.save(os.path.join(gdbDIR, 'IsoCluster'))

